# Lab 07 — Tree-based Models


> **주제:** 결정 트리부터 랜덤 포레스트, 앙상블 비교까지

---

## 학습 목표

| # | 목표 | 
|---|---| 
| 1 | 분할 기준(Gini / Entropy) 이해 및 시각화 |  
| 2 | Tic-Tac-Toe 데이터로 결정 트리 학습 · 시각화 · 가지치기 |  
| 3 | 랜덤 포레스트 — 배깅 개념 + 특성 중요도 |  
| 4 | 앙상블 비교 (Bagging vs Boosting) |  
| 5 | Exercise |  

---

**데이터셋:** UCI Tic-Tac-Toe Endgame — 틱택토 보드 상태로 X 승리 여부 이진 분류

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.patches as mpatches
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.ensemble import (
    RandomForestClassifier, BaggingClassifier,
    GradientBoostingClassifier, AdaBoostClassifier
)
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix
)

# 한글 폰트
_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

rng = np.random.default_rng(42)
print('설정 완료')

---
## Part 1. 결정 트리 — 분할 기준

### 1-1. 결정 트리란?

**결정 트리(Decision Tree)** 는 데이터를 반복적으로 분할(split)해서 클래스를 예측합니다.

```
                 [꽃잎 길이 ≤ 2.5?]
                 /              \
            예(Yes)            아니오(No)
           [Setosa]      [꽃잎 너비 ≤ 1.8?]
                          /           \
                   [Versicolor]    [Virginica]
```

각 노드에서 **어떤 특성으로 분할할지** 결정하는 기준:

| 기준 | 수식 | 특징 |
|---|---|---|
| **Gini 불순도** | $1 - \sum_k p_k^2$ | 빠름, sklearn 기본값 |
| **엔트로피** | $-\sum_k p_k \log_2 p_k$ | 정보 이론 기반, 약간 더 균형적 |

> **목표:** 분할 후 자식 노드들의 불순도 가중합을 최소화

In [ ]:
# Gini vs Entropy 시각화
p = np.linspace(1e-6, 1 - 1e-6, 300)   # 클래스 1의 비율

gini    = 2 * p * (1 - p)               # 이진 Gini: 1 - p^2 - (1-p)^2 = 2p(1-p)
entropy = -(p*np.log2(p) + (1-p)*np.log2(1-p))
error   = 1 - np.maximum(p, 1-p)        # 오분류율

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(p, gini,    color='steelblue', lw=2.5, label='Gini 불순도')
axes[0].plot(p, entropy, color='tomato',    lw=2.5, label='Entropy (÷2 정규화)')
axes[0].plot(p, error,   color='seagreen',  lw=2.0, linestyle='--', label='오분류율')
axes[0].axvline(0.5, color='gray', linestyle=':', lw=1.5)
axes[0].set_title('이진 분류에서 불순도 지표 비교')
axes[0].set_xlabel('클래스 1 비율 (p)')
axes[0].set_ylabel('불순도 / 오분류율')
axes[0].legend()

# 분할 전후 불순도 감소 예시
# 부모: [40 A, 40 B]  →  자식1: [30 A, 10 B], 자식2: [10 A, 30 B]
def gini_impurity(counts):
    total = sum(counts)
    return 1 - sum((c/total)**2 for c in counts)

splits = [
    ('나쁜 분할',   [40, 40], [20, 20], [20, 20]),
    ('보통 분할',   [40, 40], [30, 10], [10, 30]),
    ('좋은 분할',   [40, 40], [38,  2], [ 2, 38]),
    ('완벽한 분할', [40, 40], [40,  0], [ 0, 40]),
]

labels_sp, ig_vals = [], []
for name, parent, child1, child2 in splits:
    g_parent = gini_impurity(parent)
    n_tot    = sum(parent)
    n1, n2   = sum(child1), sum(child2)
    g_after  = (n1/n_tot)*gini_impurity(child1) + (n2/n_tot)*gini_impurity(child2)
    ig_vals.append(g_parent - g_after)
    labels_sp.append(name)

colors_sp = ['tomato', 'orange', 'steelblue', 'seagreen']
bars = axes[1].barh(labels_sp, ig_vals, color=colors_sp, edgecolor='k', alpha=0.85)
for bar, val in zip(bars, ig_vals):
    axes[1].text(val + 0.002, bar.get_y() + bar.get_height()/2,
                 f'{val:.3f}', va='center', fontsize=10)
axes[1].set_title('분할 방식에 따른 정보 이득(Information Gain)')
axes[1].set_xlabel('Gini 정보 이득 (클수록 좋은 분할)')
axes[1].set_xlim(0, 0.6)

plt.tight_layout()
plt.show()

print('순수 노드(한 클래스만 있을 때): Gini=0, Entropy=0  → 최적 상태')
print('완전 혼합(50:50일 때):         Gini=0.5, Entropy=1 → 최악 상태')

---
## Part 2. 결정 트리 실습 — Tic-Tac-Toe Endgame

### 2-1. 데이터셋 소개

| 속성 | 설명 |
|---|---|
| 특성 수 | 9개 (보드 9칸, 각각 `x` / `o` / `b(blank)`) |
| 샘플 수 | 958개 |
| 레이블 | `positive` (X 승리) / `negative` (X 미승리) |

```
보드 위치:
  TL | TM | TR
  ---+----+---
  ML |  C | MR
  ---+----+---
  BL | BM | BR
```

In [ ]:
# Tic-Tac-Toe 데이터 로드
try:
    ttt = fetch_openml('tic-tac-toe', version=1, as_frame=True, parser='auto')
    df  = ttt.frame.copy()
    print('OpenML에서 로드 성공')
except Exception:
    # 네트워크 없을 때: 직접 생성 (996개 유효 보드 상태 중 일부)
    import itertools
    print('OpenML 실패 → 합성 데이터 사용')
    cols  = ['TL','TM','TR','ML','C','MR','BL','BM','BR']
    lines = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
    rows, labels = [], []
    for combo in itertools.product(['x','o','b'], repeat=9):
        board = list(combo)
        x_wins = any(all(board[i]=='x' for i in line) for line in lines)
        rows.append(board)
        labels.append('positive' if x_wins else 'negative')
    df = pd.DataFrame(rows, columns=cols)
    df['class'] = labels
    df = df.sample(958, random_state=42).reset_index(drop=True)

# 컬럼명 정리
feature_cols = [c for c in df.columns if c != 'class']
print(f'특성 컬럼: {feature_cols}')
print(f'샘플 수  : {len(df)}')
print(f'클래스 분포:\n{df["class"].value_counts()}\n')
df.head(8)

In [ ]:
# 범주형 → 수치 인코딩  (x=2, o=1, b=0)
enc = OrdinalEncoder(categories=[['b','o','x']]*9)
X   = enc.fit_transform(df[feature_cols]).astype(np.float32)
y   = (df['class'] == 'positive').astype(int).values

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Train: {len(X_tr)}  |  Test: {len(X_te)}')
print(f'클래스 비율 (Train) — positive: {y_tr.mean():.2%}  negative: {1-y_tr.mean():.2%}')

# 보드 시각화 함수
def draw_board(sample_row, ax, title=''):
    board = enc.inverse_transform([sample_row])[0]
    sym_map = {'x': 'X', 'o': 'O', 'b': ''}
    colors  = {'x': '#aed6f1', 'o': '#f9caca', 'b': '#f5f5f5'}
    for i, sym in enumerate(board):
        r, c = divmod(i, 3)
        ax.add_patch(plt.Rectangle((c, 2-r), 1, 1,
                     facecolor=colors[sym], edgecolor='black', lw=1.5))
        ax.text(c+0.5, 2-r+0.5, sym_map[sym], ha='center', va='center',
                fontsize=18, fontweight='bold',
                color='steelblue' if sym=='x' else 'tomato')
    ax.set_xlim(0, 3); ax.set_ylim(0, 3)
    ax.set_aspect('equal'); ax.axis('off')
    ax.set_title(title, fontsize=9)

# 대표 샘플 보드 시각화
fig, axes = plt.subplots(1, 6, figsize=(13, 2.5))
pos_idx = np.where(y_tr == 1)[0][:3]
neg_idx = np.where(y_tr == 0)[0][:3]
for ax, idx, lbl in zip(axes[:3], pos_idx, ['X 승리']*3):
    draw_board(X_tr[idx], ax, title=lbl)
for ax, idx, lbl in zip(axes[3:], neg_idx, ['X 미승리']*3):
    draw_board(X_tr[idx], ax, title=lbl)
plt.suptitle('Tic-Tac-Toe 보드 샘플 (파랑=X, 빨강=O)', y=1.08)
plt.tight_layout()
plt.show()

### 2-2. 결정 트리 학습 및 시각화

In [ ]:
# 기본 결정 트리 (제한 없음)
dt_full = DecisionTreeClassifier(criterion='gini', random_state=42)
dt_full.fit(X_tr, y_tr)

print(f'트리 깊이     : {dt_full.get_depth()}')
print(f'리프 노드 수  : {dt_full.get_n_leaves()}')
print(f'Train Accuracy: {dt_full.score(X_tr, y_tr):.4f}')
print(f'Test  Accuracy: {dt_full.score(X_te, y_te):.4f}')

# 얕은 트리 시각화 (depth=3)
dt_viz = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
dt_viz.fit(X_tr, y_tr)

fig, ax = plt.subplots(figsize=(20, 7))
plot_tree(
    dt_viz,
    feature_names=feature_cols,
    class_names=['미승리', 'X승리'],
    filled=True,
    rounded=True,
    fontsize=10,
    ax=ax
)
ax.set_title('결정 트리 시각화 (max_depth=3) — 파란색: X승리, 주황색: 미승리')
plt.tight_layout()
plt.show()

print(f'\nmax_depth=3 트리 Test Accuracy: {dt_viz.score(X_te, y_te):.4f}')

### 2-3. 가지치기 (Pruning) — 과적합 방지

깊은 트리는 훈련 데이터에 **과적합(overfitting)** 됩니다.  
`max_depth`, `min_samples_leaf` 등으로 트리 크기를 제한합니다.

In [ ]:
# max_depth별 train / test 정확도
depths      = list(range(1, 20))
train_accs  = []
test_accs   = []

for d in depths:
    dt = DecisionTreeClassifier(criterion='gini', max_depth=d, random_state=42)
    dt.fit(X_tr, y_tr)
    train_accs.append(dt.score(X_tr, y_tr))
    test_accs.append(dt.score(X_te, y_te))

best_depth  = depths[np.argmax(test_accs)]
best_acc    = max(test_accs)

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(depths, train_accs, 'o-', color='steelblue', lw=2, label='Train Accuracy')
ax.plot(depths, test_accs,  's-', color='tomato',    lw=2, label='Test Accuracy')
ax.axvline(best_depth, color='seagreen', linestyle='--', lw=2,
           label=f'최적 깊이 = {best_depth}  (Test Acc={best_acc:.3f})')
ax.fill_between(depths, train_accs, test_accs,
                where=[tr > te for tr, te in zip(train_accs, test_accs)],
                alpha=0.15, color='tomato', label='과적합 구간')
ax.set_title('트리 깊이(max_depth)에 따른 정확도 변화')
ax.set_xlabel('max_depth')
ax.set_ylabel('Accuracy')
ax.set_ylim(0.5, 1.05)
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

print(f'깊이 제한 없음 → Train: {dt_full.score(X_tr, y_tr):.3f} / Test: {dt_full.score(X_te, y_te):.3f}  ← 과적합!')
print(f'최적 깊이({best_depth})    → Train: {train_accs[best_depth-1]:.3f} / Test: {test_accs[best_depth-1]:.3f}')

---
## Part 3. 랜덤 포레스트

### 3-1. 배깅 (Bagging) 아이디어

단일 결정 트리는 **분산(variance)이 높아** 과적합되기 쉽습니다.  
**배깅(Bootstrap Aggregating)** 은 여러 트리를 독립적으로 학습시켜 **평균(다수결)** 을 냅니다.

```
원본 데이터
    │
    ├─ 부트스트랩 샘플1 → 트리1 → 예측1 ─┐
    ├─ 부트스트랩 샘플2 → 트리2 → 예측2 ─┼─→ 다수결 → 최종 예측
    └─ 부트스트랩 샘플N → 트리N → 예측N ─┘
```

**랜덤 포레스트** = 배깅 + **무작위 특성 선택(Feature Subsampling)**
- 각 분할 시 전체 특성 중 $\sqrt{d}$개만 무작위로 선택
- 트리 간 상관성을 낮춰 분산 감소 효과 극대화

In [ ]:
# 트리 수(n_estimators)에 따른 랜덤 포레스트 성능
n_trees_list = [1, 5, 10, 20, 50, 100, 200]
rf_train_accs, rf_test_accs = [], []

for n in n_trees_list:
    rf = RandomForestClassifier(n_estimators=n, random_state=42, n_jobs=-1)
    rf.fit(X_tr, y_tr)
    rf_train_accs.append(rf.score(X_tr, y_tr))
    rf_test_accs.append(rf.score(X_te, y_te))

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(n_trees_list, rf_train_accs, 'o-', color='steelblue', lw=2, label='Train Accuracy')
ax.plot(n_trees_list, rf_test_accs,  's-', color='tomato',    lw=2, label='Test Accuracy')
ax.axhline(dt_full.score(X_te, y_te), color='gray', linestyle=':', lw=2,
           label=f'단일 트리(제한없음) Test={dt_full.score(X_te, y_te):.3f}')
ax.set_title('랜덤 포레스트 — 트리 수에 따른 정확도')
ax.set_xlabel('트리 수 (n_estimators)')
ax.set_ylabel('Accuracy')
ax.set_ylim(0.7, 1.05)
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

rf_best = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_best.fit(X_tr, y_tr)
print(f'랜덤 포레스트 (100 트리) Test Accuracy: {rf_best.score(X_te, y_te):.4f}')

### 3-2. 특성 중요도 (Feature Importance)

랜덤 포레스트는 **각 특성이 불순도를 얼마나 줄였는지** 로 중요도를 계산합니다.  
Tic-Tac-Toe에서는 어느 칸이 가장 결정적일까요?

In [ ]:
# 특성 중요도 시각화
importances = rf_best.feature_importances_
sorted_idx  = np.argsort(importances)[::-1]
sorted_feats = [feature_cols[i] for i in sorted_idx]
sorted_imps  = importances[sorted_idx]

# 중요도 막대 그래프
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

colors_imp = plt.cm.RdYlGn(sorted_imps / sorted_imps.max())
bars = axes[0].bar(sorted_feats, sorted_imps, color=colors_imp, edgecolor='k', alpha=0.85)
axes[0].set_title('랜덤 포레스트 특성 중요도 (높을수록 중요)')
axes[0].set_xlabel('보드 위치')
axes[0].set_ylabel('중요도 (Gini 감소량 평균)')
for bar, val in zip(bars, sorted_imps):
    axes[0].text(bar.get_x()+bar.get_width()/2, val+0.003,
                 f'{val:.3f}', ha='center', fontsize=8)

# 보드 위치별 중요도 히트맵
pos_map = {
    'top-left':0, 'top-middle':1, 'top-right':2,
    'middle-left':3, 'middle-middle':4, 'middle-right':5,
    'bottom-left':6, 'bottom-middle':7, 'bottom-right':8,
}
# 컬럼명이 다양할 수 있으므로 인덱스 기반으로
board_grid = importances.reshape(3, 3)
im = axes[1].imshow(board_grid, cmap='RdYlGn', vmin=0)
pos_labels = [['TL', 'TM', 'TR'], ['ML', 'C', 'MR'], ['BL', 'BM', 'BR']]
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, f'{pos_labels[i][j]}\n{board_grid[i,j]:.3f}',
                     ha='center', va='center', fontsize=11, fontweight='bold')
axes[1].set_title('보드 위치별 중요도 히트맵\n(초록=높음, 빨강=낮음)')
axes[1].set_xticks([]); axes[1].set_yticks([])
plt.colorbar(im, ax=axes[1])
plt.tight_layout()
plt.show()

print('가장 중요한 위치:', sorted_feats[0], f'(중요도: {sorted_imps[0]:.3f})')
print('→ 중앙(C)이나 모서리가 틱택토에서 가장 전략적인 위치!')

---
## Part 4. 앙상블 비교 — Bagging vs Boosting

### 4-1. 두 전략의 차이

| | **Bagging** | **Boosting** |
|---|---|---|
| 학습 방식 | 트리를 **병렬**로 독립 학습 | 트리를 **순차적**으로 학습 |
| 초점 | 분산(variance) 감소 | 편향(bias) 감소 |
| 틀린 샘플 | 균등 처리 | **가중치 증가** |
| 대표 알고리즘 | Random Forest | AdaBoost, Gradient Boosting, XGBoost |
| 과적합 위험 | 낮음 | 높음 (학습률 조정 필요) |

### 4-2. Boosting 핵심 아이디어

```
약한 분류기 1 → 오분류된 샘플 가중치 ↑
약한 분류기 2 → 오분류된 샘플 가중치 ↑  → 가중 합산 → 강한 분류기
약한 분류기 3 → ...
```

In [ ]:
# 4가지 모델 비교
models = {
    '결정 트리\n(제한없음)':      DecisionTreeClassifier(random_state=42),
    f'결정 트리\n(depth={best_depth})': DecisionTreeClassifier(max_depth=best_depth, random_state=42),
    '랜덤 포레스트\n(n=100)':     RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting\n(n=100)': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'AdaBoost\n(n=100)':          AdaBoostClassifier(n_estimators=100, random_state=42, algorithm='SAMME'),
}

results = {}
for name, model in models.items():
    model.fit(X_tr, y_tr)
    tr_acc = model.score(X_tr, y_tr)
    te_acc = model.score(X_te, y_te)
    cv_acc = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
    results[name] = dict(train=tr_acc, test=te_acc, cv5=cv_acc)
    print(f'{name.replace(chr(10)," "):<30} Train={tr_acc:.3f}  Test={te_acc:.3f}  CV-5={cv_acc:.3f}')

# 시각화
short_names = ['DT\n(무제한)', f'DT\n(d={best_depth})', 'RF\n(n=100)', 'GB\n(n=100)', 'Ada\n(n=100)']
metrics_keys = ['train', 'test', 'cv5']
labels_met   = ['Train Acc', 'Test Acc', '5-Fold CV Acc']
colors_met   = ['steelblue', 'tomato', 'seagreen']

x_pos  = np.arange(len(short_names))
width  = 0.25

fig, ax = plt.subplots(figsize=(12, 5))
for i, (key, label, color) in enumerate(zip(metrics_keys, labels_met, colors_met)):
    vals = [list(results.values())[j][key] for j in range(len(results))]
    bars = ax.bar(x_pos + (i-1)*width, vals, width, label=label,
                  color=color, edgecolor='k', alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, v+0.005,
                f'{v:.2f}', ha='center', fontsize=7.5)

ax.set_title('모델별 성능 비교 (Tic-Tac-Toe)')
ax.set_xticks(x_pos)
ax.set_xticklabels(short_names, fontsize=10)
ax.set_ylabel('Accuracy')
ax.set_ylim(0.6, 1.12)
ax.legend(loc='lower right')
ax.axhline(1.0, color='gray', linestyle=':', lw=1)
plt.tight_layout()
plt.show()

In [ ]:
# 최고 성능 모델 상세 분석
best_name  = max(results, key=lambda k: results[k]['cv5'])
best_model = list(models.values())[list(models.keys()).index(best_name)]
y_pred_best = best_model.predict(X_te)

print(f'최고 모델: {best_name.replace(chr(10), " ")}')
print()
print(classification_report(y_te, y_pred_best,
                             target_names=['미승리(0)', 'X승리(1)']))

# 혼동 행렬
cm = confusion_matrix(y_te, y_pred_best)
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['미승리', 'X승리'],
            yticklabels=['미승리', 'X승리'],
            ax=ax, linewidths=0.5)
ax.set_title(f'혼동 행렬 — {best_name.replace(chr(10), " ")}')
ax.set_xlabel('예측 클래스')
ax.set_ylabel('실제 클래스')
plt.tight_layout()
plt.show()

### 4-3. 편향-분산 트레이드오프 시각화

앙상블 방법이 어떻게 분산을 줄이는지 직관적으로 확인합니다.

In [ ]:
# 부트스트랩 샘플별 트리 예측 분산 시각화
N_BOOTSTRAP = 20
preds_single = np.zeros((N_BOOTSTRAP, len(X_te)))

for b in range(N_BOOTSTRAP):
    idx = rng.integers(0, len(X_tr), size=len(X_tr))
    dt_b = DecisionTreeClassifier(random_state=b)
    dt_b.fit(X_tr[idx], y_tr[idx])
    preds_single[b] = dt_b.predict(X_te)

# 첫 20개 테스트 샘플에 대한 예측 분포
n_show    = 20
ensemble_pred = (preds_single[:, :n_show].mean(axis=0) >= 0.5).astype(int)

fig, axes = plt.subplots(2, 1, figsize=(13, 5))

im = axes[0].imshow(preds_single[:, :n_show], cmap='RdBu', aspect='auto',
                    vmin=0, vmax=1)
axes[0].set_title(f'부트스트랩 트리 {N_BOOTSTRAP}개의 예측값 (파랑=X승리, 빨강=미승리)')
axes[0].set_xlabel('테스트 샘플 인덱스')
axes[0].set_ylabel('트리 번호')
plt.colorbar(im, ax=axes[0])

x_arr   = np.arange(n_show)
vote_ratio = preds_single[:, :n_show].mean(axis=0)
axes[1].bar(x_arr, vote_ratio, color=['steelblue' if v >= 0.5 else 'tomato' for v in vote_ratio],
            edgecolor='k', alpha=0.85)
axes[1].axhline(0.5, color='black', lw=1.5, linestyle='--', label='임계값 0.5')
for i, (v, gt) in enumerate(zip(vote_ratio, y_te[:n_show])):
    marker = '✓' if (v>=0.5)==gt else '✗'
    axes[1].text(i, v + 0.04, marker, ha='center', fontsize=9,
                 color='green' if marker=='✓' else 'red')
axes[1].set_title('앙상블 투표 비율 (✓=정답, ✗=오답)')
axes[1].set_xlabel('테스트 샘플')
axes[1].set_ylabel("'X승리' 투표 비율")
axes[1].set_ylim(0, 1.25)
axes[1].legend()

plt.tight_layout()
plt.show()

single_accs = [(preds_single[b, :] == y_te).mean() for b in range(N_BOOTSTRAP)]
print(f'개별 트리 평균 정확도: {np.mean(single_accs):.3f} ± {np.std(single_accs):.3f}')
print(f'앙상블 (다수결) 정확도: {(ensemble_pred == y_te[:n_show]).mean():.3f}')

---
## Exercise

### Exercise 1. Gini vs Entropy 분할 기준 비교

`criterion='gini'`와 `criterion='entropy'` 두 가지 기준으로 결정 트리를 학습하고,  
`max_depth=1`부터 `15`까지 Test Accuracy를 비교하는 그래프를 그리세요.  
어느 기준이 이 데이터에 더 적합한가요?

In [ ]:
# Exercise 1: Gini vs Entropy 비교
# Your code here

### Exercise 2. `min_samples_leaf`로 가지치기

`max_depth` 외에 `min_samples_leaf` (리프 노드의 최소 샘플 수) 도 가지치기에 사용됩니다.  
`min_samples_leaf = 1, 2, 5, 10, 20, 50` 각각에 대해 Train/Test Accuracy를 비교하고,  
트리 크기(리프 수)도 함께 시각화하세요.

In [ ]:
# Exercise 2: min_samples_leaf 실험
# Your code here

### Exercise 3. (도전) 랜덤 포레스트 하이퍼파라미터 탐색

아래 하이퍼파라미터 조합으로 **그리드 탐색**을 수행하고 최적 조합을 찾으세요.

```python
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth':    [None, 5, 10],
    'max_features': ['sqrt', 'log2'],
}
```

힌트: `sklearn.model_selection.GridSearchCV`를 사용하세요.  
5-fold 교차 검증 기준 최고 Test Accuracy와 파라미터 조합을 출력하세요.

In [ ]:
# Exercise 3: GridSearchCV
# Your code here

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **Gini 불순도** | $1 - \sum p_k^2$ — 분할 품질 측정, sklearn 기본값 |
| **엔트로피** | $-\sum p_k \log_2 p_k$ — 정보 이론 기반 분할 기준 |
| **정보 이득** | 부모 불순도 − 자식 가중 불순도 — 클수록 좋은 분할 |
| **가지치기** | `max_depth`, `min_samples_leaf` 등으로 과적합 방지 |
| **배깅** | 부트스트랩 샘플로 트리 병렬 학습 후 다수결 |
| **랜덤 포레스트** | 배깅 + 무작위 특성 선택 → 분산 감소 |
| **특성 중요도** | Gini 감소량 평균 — 어떤 특성이 결정적인지 파악 |
| **부스팅** | 오분류 샘플에 가중치 증가, 순차 학습 → 편향 감소 |
| **Gradient Boosting** | 잔차(residual)를 반복 학습하는 강력한 앙상블 |
| **편향-분산** | 단일 트리=고분산, 앙상블=분산 감소, 부스팅=편향 감소 |

---

**다음 강의 (Week 8):** 신경망 — 다층 퍼셉트론, 활성화 함수, 역전파 알고리즘